---
title: "Selection Bias & Missing Data Challenge"
subtitle: "Creating a Statistics Meme: Write Your Own Functions"
format:
  html: default
execute:
  echo: false
  eval: true
---

# 🎨 Selection Bias & Missing Data Challenge

::: {.callout-important}
## 📊 Challenge Overview

**Your Task:** Create a four-panel statistics meme demonstrating selection bias. You'll write three Python functions yourself to complete the workflow, then assemble them into a professional meme.

**Deliverables:**

1. Three Python functions you write yourself:
   - `step4_create_block_letter.py` - Create a block letter "S" matching image dimensions
   - `step5_create_masked.py` - Apply the letter mask to the stippled image
   - `create_meme.py` - Assemble all components into the four-panel meme
2. A complete `index.qmd` file that uses all functions to generate your meme
3. Your final statistics meme (as a PNG file) using your own image

**Key Learning:** This challenge teaches you to write modular Python functions and assemble them into a complete workflow. You'll learn to structure code professionally and create a memorable visual representation of selection bias.

**Repository Information:**

- **Source/Starter File:** Available in the main repository at [https://github.com/flyaflya/selectionBiasChallenge](https://github.com/flyaflya/selectionBiasChallenge)
- **Challenge Read Online:** View the challenge instructions at [https://flyaflya.github.io/selectionBiasChallenge](https://flyaflya.github.io/selectionBiasChallenge)
- **Your Submission:** Fork this challenge and create your GitHub Pages site at `https://[your-username].github.io/selectionBiasChallenge/`
:::

## The Problem: Visualizing Selection Bias

Selection bias occurs when observed data isn't representative of the population. Your meme will demonstrate this concept through visual metaphor:

- **Reality**: Your original image represents the true population
- **Your Model**: Your stippled image represents your data collection (sampling)
- **Selection Bias**: A bold letter "S" represents a systematic pattern of missing data
- **Estimate**: Stippled image with "S" mask applied shows the biased estimate—what you see when selection bias removes data points in a systematic pattern

**Key Concept:** Images are simply matrices—2D arrays where each value represents a pixel (0.0 = black, 1.0 = white). Your stippled image is a matrix with black dots (data points) on a white background. Selection bias removes some of these pixels (data points) in a systematic pattern (the "S"), creating a biased estimate.

**Key Insight:** When data is missing in a systematic pattern (not random), your estimates become biased. The "S" shape makes it visually obvious that the missing data follows a pattern, just like real selection bias in statistics.

## Step 1: Prepare Image

Load an image, convert to grayscale, and resize to appropriate dimensions while maintaining aspect ratio.

In [ ]:
#| label: step1-prepare
#| echo: false
#| fig-cap: Original image prepared for processing

import numpy as np
import matplotlib.pyplot as plt
from step1_prepare_image import prepare_image

# Load and prepare the image
# CHANGE THIS to use your own image!
img_path = 'rohith.jpg'  # Example image - replace with your own image
gray_image = prepare_image(img_path, max_size=512)

# Display the prepared image
fig, ax = plt.subplots(figsize=(6.5, 5))
ax.imshow(gray_image, cmap='gray', vmin=0, vmax=1)
ax.axis('off')
ax.set_title('Step 1: Prepared Image', fontsize=14, fontweight='bold', pad=10)
plt.tight_layout()
plt.show()

## Step 2: Create Stippled Image

Generate a blue noise stippling pattern from the prepared image. This creates a pattern of dots that preserves visual information while maintaining good spatial distribution.

In [ ]:
#| label: step2-stipple
#| echo: false
#| fig-cap: Blue noise stippling pattern
#| warning: false

from step2_create_stipple import create_stipple

# Create stippled image
stipple_pattern, samples = create_stipple(
    gray_image,
    percentage=0.08,  # 8% of pixels will be stippled
    sigma=0.9,  # Repulsion radius
    content_bias=0.9,  # Strongly follow importance map
    noise_scale_factor=0.1,  # Moderate exploration
    extreme_downweight=0.5,  # Moderate downweighting of extremes
    extreme_threshold_low=0.2,  # Downweight tones below 0.2
    extreme_threshold_high=0.8,  # Downweight tones above 0.8
    extreme_sigma=0.1  # Smooth transition width
)

# Display the stippled image
fig, ax = plt.subplots(figsize=(6.5, 5))
ax.imshow(stipple_pattern, cmap='gray', vmin=0, vmax=1)
ax.axis('off')
ax.set_title('Step 2: Stippled Image', fontsize=14, fontweight='bold', pad=10)
plt.tight_layout()
plt.show()

## Step 3: Create Tonal Analysis (Optional Refinement Step)

Create a tonal analysis by dividing the image into a grid and computing average brightness in each section. This visualizes the distribution of tones and helps identify which brightness ranges are most important.

In [ ]:
#| label: step3-tonal
#| echo: false
#| fig-cap: Box-averaged tonal analysis showing brightness distribution

from step3_create_tonal import create_tonal
import matplotlib.pyplot as plt

# Create tonal analysis with a 16×12 grid
grid_rows = 16
grid_cols = 12
tonal_image, average_tones, tonal_stats = create_tonal(
    gray_image,
    grid_rows=grid_rows,
    grid_cols=grid_cols,
    return_full_image=True
)

# Display the box-averaged tonal image with text annotations
fig, ax = plt.subplots(figsize=(6.5, 5))

# Show box-averaged tonal image
ax.imshow(tonal_image, cmap='gray', vmin=0, vmax=1)
ax.axis('off')
ax.set_title('Step 3: Box-Averaged Tonal Analysis', fontsize=14, fontweight='bold', pad=10)

# Calculate grid cell dimensions for text placement
h, w = gray_image.shape
section_h = h / grid_rows
section_w = w / grid_cols

# Add text annotations showing tone values (2 decimals) at the center of each grid cell
for i in range(grid_rows):
    for j in range(grid_cols):
        tone = average_tones[i, j]
        # Calculate center position of the grid cell
        y_center = (i + 0.5) * section_h
        x_center = (j + 0.5) * section_w
        # Use white text for dark sections, black text for light sections
        text_color = 'white' if tone < 0.5 else 'black'
        ax.text(x_center, y_center, f'{tone:.2f}', 
                ha='center', va='center', 
                color=text_color, fontsize=6, fontweight='bold')

plt.tight_layout()
plt.show()

# Print key statistics for parameter tuning
print(f"\n📊 Tonal Statistics for Parameter Tuning:")
print(f"  Mean brightness: {tonal_stats['mean']:.3f}")
print(f"  Standard deviation: {tonal_stats['std']:.3f}")
print(f"  Brightness range: [{tonal_stats['min']:.3f}, {tonal_stats['max']:.3f}]")

## Step 4: Create Block Letter "S" 
block letter "S" is created using the create_block_letter_s function.

In [ ]:
#| label: step4-block-letter
#| echo: false
#| fig-cap: Block letter S representing selection bias

from step4_create_block_letter import create_block_letter_s

# Get image dimensions
h, w = gray_image.shape

# Create block letter S
block_letter = create_block_letter_s(h, w, letter="S", font_size_ratio=0.9)

# Display the block letter
fig, ax = plt.subplots(figsize=(6.5, 5))
ax.imshow(block_letter, cmap='gray', vmin=0, vmax=1)
ax.axis('off')
ax.set_title('Step 4: Selection Bias (Block Letter S)', fontsize=14, fontweight='bold', pad=10)
plt.tight_layout()
plt.show()

## Step 5: Create Masked Image 
Block letter mask is applied on the stippled image.

In [ ]:
#| label: step5-masked
#| echo: false
#| fig-cap: Masked stippled image showing selection bias effect

from step5_create_masked import create_masked_stipple

# Create masked stippled image
masked_stipple = create_masked_stipple(
    stipple_pattern,
    block_letter,
    threshold=0.5  # Pixels below 0.5 are considered part of the mask
)

# Display the masked image
fig, ax = plt.subplots(figsize=(6.5, 5))
ax.imshow(masked_stipple, cmap='gray', vmin=0, vmax=1)
ax.axis('off')
ax.set_title('Step 5: Masked Stippled Image (Estimate)', fontsize=14, fontweight='bold', pad=10)
plt.tight_layout()
plt.show()

## Create the Final Statistics Meme 

In [ ]:
#| label: create-final-meme
#| echo: false

from create_meme import create_statistics_meme

# Create the final meme
create_statistics_meme(
    original_img=gray_image,
    stipple_img=stipple_pattern,
    block_letter_img=block_letter,
    masked_stipple_img=masked_stipple,
    output_path="statistics_meme.png",
    dpi=150,
    background_color="lightgray"  # or "pink", "lightgray", etc.
)

## Final Statistics Meme

Here's my completed statistics meme demonstrating selection bias:

In [ ]:
#| label: final-meme
#| echo: false
#| fig-cap: Statistics meme demonstrating selection bias

# Display the final meme
from IPython.display import Image, display
display(Image("statistics_meme.png"))

### Explanation

 - This meme demonstrates selection bias by showing how selectively keeping only certain stipple points distorts our understanding of the original image. The Reality panel reflects the true population, while the stippled image (Model) represents the full dataset we collect. When the selection bias removes all data except the points inside the “S,” the resulting Estimate becomes systematically distorted and no longer represents the true population — just like biased sampling in real studies leads to incorrect conclusions.